In [1]:
import mido
from mido import MidiFile, Message, MidiTrack
import torch
import os
import numpy as np
import time
import midi
import glob  
from tqdm import tqdm

os.chdir('D:\engsci\year 3\CLASS\APS360\Project') #Sets current working directory!
# from midiutil.MidiFile import MIDIFile

# mid.save('songname_mid.mid') to save songs
Process: 
1) Extract the original time signatures from the data by subtracting times...
2) Remove the sustain column of the array.
2) Compile into a MIDI file
#from sustain to velocity zero.
MIDIUtil: https://code.google.com/archive/p/midiutil/
Other sources: https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c
^^MIDIUtil has a function that can just about take our array as an input. However, we will do it the trickier way with Mido because it's easier to use with Colab. Specifically:

run: 

pip install mido

on its own line. Then run:

import mido

in the next code block.

Need also: https://pypi.org/project/py-midi/

pip install py-midi

then:

import midi

In [2]:
#Essential workflow is: Loop through array.
#For each note, extract the sustain value of the note.
#Go to next note. If time to next note is smaller than sustain, add next note and call the function again with updated sustain as sustain-(time of next note)
#If time to next note is larger than sustain, add the original note with velocity zero and time = sustain.

def Numpy2Midi(array):                      # Defining it iteratively...
    #Start with initializing a new Mido Track:
    mid = MidiFile()
    track = MidiTrack()
    already_added = []
    for i,note in enumerate(array):         # Get the index and the note. Array must be int array             
        #print(note)
        sustain = note[3]                   # Get how long the note is on for.
        #print(sustain)
        j = 1                               #
        if i not in already_added:
            track.append(Message('note_on',note = array[i,0], velocity = array[i,1],time = array[i,2])) # Add the note start to the track.
        while (i+j <= array.shape[0]-1) and (sustain > array[i+j,2]):       
            # ^^ If sustain is longer than the time to the next note or we reached the last note...
            if (i+j) not in already_added:
#                 print('not already added')
                track.append(Message('note_on',note = array[i+j,0], velocity = array[i+j,1],time = array[i+j,2]))
                already_added.append([i+j]) #Remember that we already added the start of this note.
#                 print('updated already_apped list')
            sustain = sustain - array[i+j,2] # Subtract the time to the next note.
            j += 1 #update index to check agains.
        track.append(Message('note_on',note = array[i,0], velocity = 0,time = sustain)) # Add the off note.
    mid.tracks.append(track)
    return mid

In [3]:
#load in the cleanNotes file:
import numpy
cleanNotes = numpy.load('cleanNotes.npy')
print(cleanNotes)
cleanNotes = cleanNotes.astype('int')
print(cleanNotes)
print(cleanNotes.shape[0]-1)

[[ 67.  74. 762. 749.]
 [ 39.  53.  15.  82.]
 [ 63.  55.   5. 412.]
 ...
 [ 82.  92.   1. 472.]
 [ 79.  95.   1. 468.]
 [ 39.  90.   3. 437.]]
[[ 67  74 762 749]
 [ 39  53  15  82]
 [ 63  55   5 412]
 ...
 [ 82  92   1 472]
 [ 79  95   1 468]
 [ 39  90   3 437]]
3992


In [4]:
test = Numpy2Midi(cleanNotes)
test.save('test_song_back.mid')